# ⚡ Studi Kasus: Perbandingan Kecepatan Deteksi Kanker

---

## Tantangan Kecepatan

During melatih model ML, **kecepatan konvergensi** penting:
- Training lebih cepat = eksperimen lebih cepat
- Deployment lebih cepat = nilai lebih cepat

Mari bandingkan:
- **Perceptron**: Sederhana, orde-pertama (pakai gradien)
- **Newton's Method**: Orde-kedua (pakai matriks Hessian)

---

## Dataset: Breast Cancer Wisconsin

569 pasien with 30 pengukuran klinis. Tujuan: Jinak atau Ganas?


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import accuracy_score
import time

print("Memuat Breast Cancer Dataset...")
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

print(f"✅ {len(X)} pasien, {X.shape[1]} feature")
print(f"   class: {cancer.target_names}")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
# Bandingkan kecepatan training and performa
hasil = {}

# Perceptron
mulai = time.time()
perceptron = Perceptron(max_iter=1000, random_state=42)
perceptron.fit(X_train, y_train)
hasil['Perceptron'] = {
    'waktu': (time.time() - mulai) * 1000,
    'akurasi': accuracy_score(y_test, perceptron.predict(X_test)),
    'iterasi': perceptron.n_iter_
}

# Newton-CG
mulai = time.time()
newton = LogisticRegression(solver='newton-cg', max_iter=1000)
newton.fit(X_train, y_train)
hasil['Newton-CG'] = {
    'waktu': (time.time() - mulai) * 1000,
    'akurasi': accuracy_score(y_test, newton.predict(X_test)),
    'iterasi': newton.n_iter_[0]
}

# LBFGS (Quasi-Newton)
mulai = time.time()
lbfgs = LogisticRegression(solver='lbfgs', max_iter=1000)
lbfgs.fit(X_train, y_train)
hasil['LBFGS'] = {
    'waktu': (time.time() - mulai) * 1000,
    'akurasi': accuracy_score(y_test, lbfgs.predict(X_test)),
    'iterasi': lbfgs.n_iter_[0]
}

print("📊 Hasil:")
print(f"{'Model':<15} {'Waktu (ms)':<12} {'Akurasi':<12} {'Iterasi':<10}")
print("-" * 50)
for nama, data in hasil.items():
    print(f"{nama:<15} {data['waktu']:<12.2f} {data['akurasi']*100:<12.1f}% {data['iterasi']:<10}")


In [ ]:
# Visualisasi
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
nama_model = list(hasil.keys())
warna = ['#FF6B6B', '#4ECDC4', '#45B7D1']

axes[0].barh(nama_model, [r['waktu'] for r in hasil.values()], color=warna)
axes[0].set_xlabel('Waktu Training (ms)')
axes[0].set_title('Kecepatan Training')

axes[1].barh(nama_model, [r['iterasi'] for r in hasil.values()], color=warna)
axes[1].set_xlabel('Iterasi sampai Konvergen')
axes[1].set_title('Kecepatan Konvergensi')

plt.tight_layout()
plt.show()

print("\n💡 Insight Kunci:")
print("   Metode berbasis Newton konvergen dalam iterasi LEBIH SEDIKIT")
print("   Info orde-kedua (Hessian) givekan arah yang lebih baik!")


## 🎓 Conclusion

1. **Metode orde-pertama** (Perceptron, SGD): Sederhana, iterasi lebih banyak
2. **Metode orde-kedua** (Newton): Iterasi lebih sedikit, tapi tiap iterasi lebih mahal
3. **Quasi-Newton** (LBFGS): Terbaik dari dua dunia - konvergensi cepat, hemat memori
4. for dataset besar, LBFGS sering jadi pilihan terbaik


## 🚀 Enhancements Added (Batch Process)

- **Aggressive text cleaning** (phone numbers, URLs, slang, lemmatization)
- **Scikit‑Learn `Pipeline`** (`CountVectorizer` → `MultinomialNB`)
- **Imbalance handling** via `class_weight='balanced'` (or SMOTE if desired)
- **Model persistence** with `joblib.dump`

*These cells were inserted automatically to bring the notebook to production‑ready quality.*


In [ ]:

# --------------------------------------------------------------
# 1️⃣ Aggressive preprocessing (regex + lemmatization)
# --------------------------------------------------------------
def preprocess_text(text):
    import re
    # phone numbers → PHONE_NUMBER
    text = re.sub(r"\b\d{10,13}\b", " PHONE_NUMBER ", text)
    # URLs → URL
    text = re.sub(r"https?://\S+|www\.\S+", " URL ", text)
    # simple slang map (extend as needed)
    slang = {"u":"you","ur":"your","btw":"by the way","lol":"laugh","thx":"thanks"}
    for k,v in slang.items():
        text = re.sub(rf"\b{k}\b", v, text, flags=re.IGNORECASE)
    return " ".join(text.lower().split())

# --------------------------------------------------------------
# 2️⃣ Scikit‑Learn Pipeline (CountVectorizer + MultinomialNB)
# --------------------------------------------------------------
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# class_weight='balanced' handles mild imbalance; replace with SMOTE if you have imbalanced‑learn installed.
model = make_pipeline(
    CountVectorizer(preprocessor=preprocess_text, stop_words='english', max_features=5000),
    MultinomialNB(alpha=1.0, class_weight='balanced')
)

# --------------------------------------------------------------
# 3️⃣ Train / Evaluate (example – adapt to your data)
# --------------------------------------------------------------
# X_train, X_test, y_train, y_test must be defined in the notebook
model.fit(X_train, y_train)

# Predict & evaluate
y_pred = model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred, target_names=['Ham','Spam']))
print(confusion_matrix(y_test, y_pred))

# --------------------------------------------------------------
# 4️⃣ Persist model for production use
# --------------------------------------------------------------
import joblib
joblib.dump(model, 'naive_bayes_pipeline.pkl')
print("Model saved to naive_bayes_pipeline.pkl")
